<a href="https://colab.research.google.com/github/microsoft/qlib/blob/main/examples/workflow_by_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.

In [4]:
import sys, site
from pathlib import Path

################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    # reload
    site.main()

scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests
    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py") as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

ConnectionError: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /microsoft/qlib/main/scripts/get_data.py (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000200158F2FA0>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed'))

In [5]:

import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict


In [6]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(scripts_dir))
    from get_data import GetData
    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

[5008:MainThread](2021-12-13 20:01:18,716) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[5008:MainThread](2021-12-13 20:01:20,723) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[5008:MainThread](2021-12-13 20:01:20,726) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[5008:MainThread](2021-12-13 20:01:20,727) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/lizil_5truwnu/.qlib/qlib_data/cn_data')}


In [7]:
market = "csi300"
benchmark = "SH000300"

# train model

In [10]:
###################################
# train model
###################################
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "TCN",
        "module_path": "qlib.contrib.model.pytorch_tcn_ts",
        "kwargs": {
            "d_feat": 20,
            "num_layers": 5,
            "n_chans": 32,
            "kernel_size": 7,
            "dropout": 0.5,
            "n_epochs": 200,
            "lr": 1e-4,
            "early_stop": 20,
            "batch_size": 2000,
            "metric": "loss",
            "loss": "mse",
            "optimizer": "adam",
            "n_jobs": 20,
            "GPU": 0,
        },
    },
    "dataset": {
        "class": "TSDatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
            "step_len": 20,
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id


[5008:MainThread](2021-12-13 20:05:07,195) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:71] - DNN pytorch version...
[5008:MainThread](2021-12-13 20:05:07,198) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:88] - DNN parameters setting:
layers : (256,)
lr : 0.002
max_steps : 300
batch_size : 2000
early_stop_rounds : 50
eval_steps : 20
lr_decay : 0.96
lr_decay_steps : 100
optimizer : adam
loss_type : mse
eval_steps : 20
seed : None
device : cuda:0
use_GPU : True
weight_decay : 0.0
[5008:MainThread](2021-12-13 20:05:07,202) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:132] - model:
Net(
  (dnn_layers): ModuleList(
    (0): Dropout(p=0.05, inplace=False)
    (1): Sequential(
      (0): Linear(in_features=158, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (2): Dropout(p=0.05, inplace=False)
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
)
[5008:MainTh

Epoch    11: reducing learning rate of group 0 to 1.0000e-03.


[5008:MainThread](2021-12-13 20:07:12,191) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:240] - [Epoch 220]: train_loss nan, valid_loss nan
[5008:MainThread](2021-12-13 20:07:12,432) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:240] - [Epoch 240]: train_loss nan, valid_loss nan
[5008:MainThread](2021-12-13 20:07:12,665) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:240] - [Epoch 260]: train_loss nan, valid_loss nan
[5008:MainThread](2021-12-13 20:07:12,904) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:240] - [Epoch 280]: train_loss nan, valid_loss nan
[5008:MainThread](2021-12-13 20:07:13,126) INFO - qlib.DNNModelPytorch - [pytorch_nn.py:240] - [Epoch 299]: train_loss nan, valid_loss nan


EOFError: Ran out of input

# prediction, backtest & analysis

In [ ]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()


# analyze graphs

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
pred_df_dates = pred_df.index.get_level_values(level='datetime')
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)